In [1]:
!pip install scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 786.4/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 870.1 kB/s  0:00:00


In [1]:
# ===========================================
# SISTEMA EXPERTO DIFUSO - DETECCIÓN DE TERREMOTOS
# Modelo Mamdani implementado con scikit-fuzzy
# ===========================================

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# ------------------------------------------------------
# 1. Definición de las variables de entrada y salida
# ------------------------------------------------------

# Entradas:
aceleracion = ctrl.Antecedent(np.arange(0, 1.1, 0.01), 'aceleracion')   # g (0–1g)
frecuencia  = ctrl.Antecedent(np.arange(0, 20.1, 0.1), 'frecuencia')    # Hz
duracion    = ctrl.Antecedent(np.arange(0, 60.1, 1), 'duracion')        # segundos
snr         = ctrl.Antecedent(np.arange(0, 30.1, 0.1), 'snr')           # dB

# Salida:
amenaza = ctrl.Consequent(np.arange(0, 11, 1), 'amenaza')               # 0–10 escala de riesgo

# ------------------------------------------------------
# 2. Definición de las funciones de pertenencia
# ------------------------------------------------------

# Aceleración (g)
aceleracion['baja']  = fuzz.trimf(aceleracion.universe, [0,   0,   0.3])
aceleracion['media'] = fuzz.trimf(aceleracion.universe, [0.2, 0.5, 0.8])
aceleracion['alta']  = fuzz.trimf(aceleracion.universe, [0.6, 1,   1])

# Frecuencia (Hz)
frecuencia['baja']  = fuzz.trapmf(frecuencia.universe,  [0, 0, 2, 5])
frecuencia['media'] = fuzz.trapmf(frecuencia.universe,  [3, 6, 10, 13])
frecuencia['alta']  = fuzz.trapmf(frecuencia.universe,  [11, 15, 20, 20])

# Duración (s)
duracion['corta'] = fuzz.trimf(duracion.universe, [0,  0,  10])
duracion['media'] = fuzz.trimf(duracion.universe, [5,  20, 35])
duracion['larga'] = fuzz.trimf(duracion.universe, [30, 60, 60])

# Amenaza sísmica (0–10)
amenaza['sin_actividad'] = fuzz.trimf(amenaza.universe, [0,  0,  2])
amenaza['microtemblor']  = fuzz.trimf(amenaza.universe, [1,  3,  5])
amenaza['leve']          = fuzz.trimf(amenaza.universe, [4,  5,  6])
amenaza['moderado']      = fuzz.trimf(amenaza.universe, [5,  7,  8])
amenaza['fuerte']        = fuzz.trimf(amenaza.universe, [7,  9,  10])
amenaza['destructivo']   = fuzz.trimf(amenaza.universe, [9,  10, 10])

# Signal-to-Noise Ratio (dB)
snr['pobre']      = fuzz.trapmf(snr.universe, [0,  0,  5,  10])
snr['aceptable']  = fuzz.trapmf(snr.universe, [8,  12, 18, 22])
snr['buena']      = fuzz.trapmf(snr.universe, [20, 25, 30, 30])

# ------------------------------------------------------
# 3. Definición de las reglas difusas
# ------------------------------------------------------

# Reglas originales
rule1 = ctrl.Rule(aceleracion['baja']  & frecuencia['alta']  & duracion['corta'], amenaza['sin_actividad'])
rule2 = ctrl.Rule(aceleracion['media'] & frecuencia['media'] & duracion['media'], amenaza['leve'])
rule3 = ctrl.Rule(aceleracion['alta']  & frecuencia['baja']  & duracion['larga'], amenaza['fuerte'])
rule4 = ctrl.Rule(aceleracion['alta']  & frecuencia['baja']  & duracion['media'], amenaza['moderado'])
rule5 = ctrl.Rule(aceleracion['media'] & frecuencia['baja']  & duracion['larga'], amenaza['moderado'])
rule6 = ctrl.Rule(aceleracion['alta']  & frecuencia['baja']  & duracion['larga'], amenaza['destructivo'])
rule7 = ctrl.Rule(aceleracion['baja']  & frecuencia['media'] & duracion['corta'], amenaza['microtemblor'])

# Reglas que incorporan SNR (para que el sistema admita la entrada 'snr' y la use)
# - Si la SNR es pobre, la medición es poco fiable: tendencia a "sin_actividad".
rule_snr1 = ctrl.Rule(snr['pobre'], amenaza['sin_actividad'])
# - Si la SNR es aceptable y la aceleración no es alta, tendencia a "microtemblor".
rule_snr2 = ctrl.Rule(snr['aceptable'] & (aceleracion['baja'] | aceleracion['media']), amenaza['microtemblor'])
# - Si la SNR es buena y hay aceleración media/alta, refuerza "moderado".
rule_snr3 = ctrl.Rule(snr['buena'] & (aceleracion['media'] | aceleracion['alta']), amenaza['moderado'])

# ------------------------------------------------------
# 4. Creación del sistema de control difuso
# ------------------------------------------------------
control_sismo = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7,
    rule_snr1, rule_snr2, rule_snr3
])
simulador = ctrl.ControlSystemSimulation(control_sismo)

# ------------------------------------------------------
# 5. Ejemplo de simulación
# ------------------------------------------------------

# === Entrada de datos (en Colab/terminal) ===
valor_acel = float(input("Ingrese la aceleración del terreno (g, 0–1): "))
valor_freq = float(input("Ingrese la frecuencia dominante (Hz, 0–20): "))
valor_dur  = float(input("Ingrese la duración del evento (s, 0–60): "))
valor_snr  = float(input("Ingrese la SNR (dB, 0–30): "))

# Asignar y calcular
simulador.input['aceleracion'] = valor_acel
simulador.input['frecuencia']  = valor_freq
simulador.input['duracion']    = valor_dur
simulador.input['snr']         = valor_snr

simulador.compute()

print(f"\n🔍 Nivel de amenaza sísmica: {simulador.output['amenaza']:.2f} / 10")

# ------------------------------------------------------
# 6. Visualización opcional de las funciones
# ------------------------------------------------------
# Nota: estas ventanas requieren entorno gráfico.
#aceleracion.view()
#frecuencia.view()
#duracion.view()
#snr.view()
# amenaza.view(simulador)



🔍 Nivel de amenaza sísmica: 0.67 / 10
